### Least Squares

##### Standard Problem:
$$
\begin{aligned}
\min_{x} \quad & \lVert x \rVert_2^2 \\
\textrm{s.t.} \quad & Ax = b
\end{aligned}
$$

##### Lagrangian:
$$
\begin{aligned}
\mathcal{L}(x, \lambda) = \min_{x} \max_{\lambda} \quad & x^{\top}x + \lambda^{\top} (Ax - b)
\end{aligned}
$$

##### Analytical Solution
$$
\begin{aligned}
\frac{\partial}{\partial \lambda} \rightarrow & Ax^\ast = b (1) \\
\frac{\partial}{\partial x} \rightarrow & 2x^\ast + A^\top \lambda^\ast = 0 \\
& x^\ast = - \frac{1}{2}A^\top\lambda^\ast (2)
\end{aligned}
$$

Replace $x^\ast$ in $(1)$
$$
\begin{aligned}
A( - \frac{1}{2}A^\top\lambda^\ast) & = b \\
 - \frac{1}{2}AA^\top\lambda^\ast & = b \\
 \lambda^\ast = -2(AA^\top)^{-1}b
\end{aligned}
$$

Replace $\lambda^\ast$ in $(2)$
$$
x^\ast = A^\top(AA^\top)^{-1}b
$$